### This jupyter notebook is a solution to the kaggle competition of time series forecasting where we have to predict future sales using various parameters and forecasting techniques 

In [43]:
# Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import os
from sklearn.model_selection import train_test_split, GridSearchCV # Used to find ideal hyperparameters for tuning
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
# StandardScaler converts mean to 0 and deviation to 1, this helps in ensuring similar scale
# OrdinalEncoder converts categorical data to numeric data while preserving ranking
# LabelEncoder converts categorical data to numeric data
# OneHotEncoder transforms categorical variables into a binary vector where each category is represented as 0 or 1
from sklearn.impute import SimpleImputer, KNNImputer
# SimpleImputer is used to handle missing values
# KNNImputer is used to handle missing values using nearest neighbours
from sklearn.base import BaseEstimator, TransformerMixin
# BaseEstimator is used to make custom machine learning models
# TransformerMixin is used to create custom transformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
# ColumnTransformer applies different transformers to different columns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

%matplotlib.inline

UsageError: Line magic function `%matplotlib.inline` not found.


In [ ]:
# Importing the data
train = pd.read_csv('train.csv')
holiday_events = pd.read_csv('holidays_events.csv')
oil = pd.read_csv('oil.csv')
stores = pd.read_csv('stores.csv')
transactions = pd.read_csv('transactions.csv')

In [ ]:
# Having a look at the train dataset
train.head()

In [ ]:
# Having a look at the oil dataset
oil.head()

In [ ]:
# Having a look at the stores dataset
stores.head()

In [ ]:
# Having a look at the transactions dataset
transactions.head()

In [ ]:
# Having a look at the holiday_events dataset
pd.read_csv('holidays_events.csv')

In [ ]:
# Merging all the datasets into train dataset
train_full = train \
  .merge(holiday_events, how = 'left', on = ['date']).rename(columns={'type':'holiday_type'}) \
  .merge(stores, how = 'left', on = ['store_nbr']).rename(columns={'type':'city_type'}) \
  .merge(transactions, how = 'left', on = ['store_nbr','date']) \
  .merge(oil, how = 'left', on = ['date'] )

In [44]:
# Having a look at the new combined dataset
train_full.head()

,id,date,store_nbr,family,sales,onpromotion,holiday_type,locale,locale_name,description,transferred,city,state,city_type,cluster,transactions,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN,NaN


In [45]:
# Importing the test dataset
test = pd.read_csv('test.csv')

In [46]:
# Having a look at the test dataset
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [47]:
# Merging all datasets into test dataset
test_full = test \
  .merge(holiday_events, how = 'left', on = ['date']).rename(columns={'type':'holiday_type'}) \
  .merge(stores, how = 'left', on = ['store_nbr']).rename(columns={'type':'city_type'}) \
  .merge(transactions, how = 'left', on = ['store_nbr','date']) \
  .merge(oil, how = 'left', on = ['date'] )

In [48]:
# Creating a new date column in the datetime format and setting it as the index
train_full['date'] = pd.to_datetime(train['date'])
train_full.set_index('date')

,id,store_nbr,family,sales,onpromotion,holiday_type,locale,locale_name,description,transferred,city,state,city_type,cluster,transactions,dcoilwtico
date,,,,,,,,,,,,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN,NaN
2013-01-01,1,1,BABY CARE,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN,NaN
2013-01-01,2,1,BEAUTY,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN,NaN
2013-01-01,3,1,BEVERAGES,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN,NaN
2013-01-01,4,1,BOOKS,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaT,3000883,9,POULTRY,438.133,0,Holiday,Local,Riobamba,Fundacion de Riobamba,False,Quito,Pichincha,B,6,2155.0,47.57
NaT,3000884,9,PREPARED FOODS,154.553,1,Holiday,Local,Riobamba,Fundacion de Riobamba,False,Quito,Pichincha,B,6,2155.0,47.57
NaT,3000885,9,PRODUCE,2419.729,148,Holiday,Local,Riobamba,Fundacion de Riobamba,False,Quito,Pichincha,B,6,2155.0,47.57


In [49]:
# Creating a custom transformer designed to drop one or more columns from the dataset
class DropColumnTransformer(BaseEstimator, TransformerMixin): # BaseEstimator provides basic functionalities like handline parameters, model validation while TransformerMixin provides methods required for transformer like fit,transform
    def __init__(self, by = 1, columns = None): # __init__ is the constructor class, by specifies how many columns to drop, columns = None indicate that all columns will be dropped on the basis of by
        self.by = by
        self.columns = columns
        
    def fit(self, X, y = None): # The fit method satisfies the transformer interface, takes input data X, target variable y
        return self
    
    def transform(self, X, y = None): # The transform method is used to satisfy the transformer interface 
        cols_to_transform = X.columns
        if self.columns:
            cols_to_transform = self.columns
        X = X.drop(cols_to_transform, axis = 1) # Specified columns are dropped
        return X # Transformed dataframe is returned

In [50]:
# Creating a custom transformer designed to convert values of columns to string format
class BoolTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, by=1, columns=None):
        self.by = by
        self.columns = columns
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        cols_to_transform = X.columns
        if self.columns:
            cols_to_transform = self.columns
        for col in cols_to_transform:
            X[col] = X[col].apply(lambda x: str(x)) # Apply function is used to perform a function on each element of a series
        return X

In [51]:
# Creating a pipeline that performs DropColumnTransformer and BoolTransformer on the full dataset
prepare_df_pipe = Pipeline( # A pipeline is used to perform multiple operations in a sequence
    steps=[
          ("drop columns", DropColumnTransformer(columns=['description','locale_name', 'id', 'sales']))
        , ("bool columns", BoolTransformer(columns=['transferred']))
    ]
)
prepare_df_pipe.fit_transform(train_full)

,date,store_nbr,family,onpromotion,holiday_type,locale,transferred,city,state,city_type,cluster,transactions,dcoilwtico
0,2013-01-01,1,AUTOMOTIVE,0,Holiday,National,False,Quito,Pichincha,D,13,NaN,NaN
1,2013-01-01,1,BABY CARE,0,Holiday,National,False,Quito,Pichincha,D,13,NaN,NaN
2,2013-01-01,1,BEAUTY,0,Holiday,National,False,Quito,Pichincha,D,13,NaN,NaN
3,2013-01-01,1,BEVERAGES,0,Holiday,National,False,Quito,Pichincha,D,13,NaN,NaN
4,2013-01-01,1,BOOKS,0,Holiday,National,False,Quito,Pichincha,D,13,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,NaT,9,POULTRY,0,Holiday,Local,False,Quito,Pichincha,B,6,2155.0,47.57
3054344,NaT,9,PREPARED FOODS,1,Holiday,Local,False,Quito,Pichincha,B,6,2155.0,47.57
3054345,NaT,9,PRODUCE,148,Holiday,Local,False,Quito,Pichincha,B,6,2155.0,47.57
3054346,NaT,9,SCHOOL AND OFFICE SUPPLIES,8,Holiday,Local,False,Quito,Pichincha,B,6,2155.0,47.57


In [52]:
# Calculating the percentage of missing values in each column
train_full.isna().sum()/len(train_full)

id              0.000000
date            0.017503
store_nbr       0.000000
family          0.000000
sales           0.000000
onpromotion     0.000000
holiday_type    0.835473
locale          0.835473
locale_name     0.835473
description     0.835473
transferred     0.835473
city            0.000000
state           0.000000
city_type       0.000000
cluster         0.000000
transactions    0.081561
dcoilwtico      0.312719
dtype: float64

In [53]:
# Replacing missing values with None and finding count of each unique value in holiday_type column
train_full['holiday_type'].fillna('None').value_counts()

None          2551824
Holiday        304722
Event           99792
Additional      67716
Transfer        16038
Work Day         8910
Bridge           5346
Name: holiday_type, dtype: int64

In [55]:
# Calculating the number of missing values in each column of the transformed dataset
prepare_df_pipe.fit_transform(train_full).isna().sum()

date              53460
store_nbr             0
family                0
onpromotion           0
holiday_type    2551824
locale          2551824
transferred           0
city                  0
state                 0
city_type             0
cluster               0
transactions     249117
dcoilwtico       955152
dtype: int64

In [56]:
# Replacing missing values with None in dcoilwtico and counting the number of unique values
train_full['dcoilwtico'].fillna('None').value_counts()

None      955152
96.44       7128
107.04      7128
46.02       7128
95.25       7128
           ...  
95.54       1782
94.35       1782
96.4        1782
93.97       1782
47.57       1782
Name: dcoilwtico, Length: 995, dtype: int64

In [57]:
# Applieng the pipeline to the complete dataset and retrieving names of columns with datatype other than object
prepare_df_pipe.fit_transform(train_full).columns[prepare_df_pipe.fit_transform(train_full).dtypes != 'object']

Index(['date', 'store_nbr', 'onpromotion', 'cluster', 'transactions',
       'dcoilwtico'],
      dtype='object')

In [58]:
# Creating multiple pipelines to encode categorical and numerical data 
none_pipe = Pipeline([ # This pipeline handles missing values in categorical features by replacing them with the 'Other' and applying one-hot encoding to those features.
    ('imputer', SimpleImputer(strategy='constant', fill_value = 'Other')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
]) 

cat_pipe = Pipeline([ # This pipeline handles missing values in categorical features by replacing them with the most frequent value and then encoding the categorical features using ordinal encoding.
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
]) 

num_pipe = Pipeline([ # This pipeline handles missing values in numerical features by replacing them with the value -1 and then scaling the numerical features using standardization.
    ('imputer', SimpleImputer(strategy='constant', fill_value = -1)),
    ('scaler', StandardScaler())
])

In [59]:
# Applieng different preprocessing steps to different columns using ColumnTransformer
preprocessor = ColumnTransformer([
      ('holiday_type, locale, transferred, city_type', none_pipe, ['holiday_type', 'locale', 'city_type', 'transferred'])
    , ('city, family', cat_pipe, ['family']) 
    , ('store_nbr, onpromotion, cluster, transactions', num_pipe, ['store_nbr', 'onpromotion', 'cluster', 'transactions'])
])

In [60]:
# Creating a pipeline that performs various preprocessing steps on different columns
pipe = Pipeline(
    steps=[
        ("drop columns", DropColumnTransformer(columns=['description','locale_name', 'id', 'city']))
      , ("bool columns", BoolTransformer(columns=['transferred']))
      , ("preprocessor", preprocessor)
      , ("model", RandomForestRegressor(n_estimators=20, verbose=1, n_jobs = -1))
    ]
)

In [61]:
# Splitting the features and the target variable
X = train_full.drop('sales', axis =1)
y = train_full['sales']

In [64]:
# Fitting the data in the final pipeline
pipe.fit(X, y)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.5min finished


Pipeline(steps=[('drop columns',
                 DropColumnTransformer(columns=['description', 'locale_name',
                                                'id', 'city'])),
                ('bool columns', BoolTransformer(columns=['transferred'])),
                ('preprocessor',
                 ColumnTransformer(transformers=[('holiday_type, locale, '
                                                  'transferred, city_type',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='Other',
                                                                                 strategy='constant')),
                                                                  ('encoder',...
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  

In [65]:
# Making the predictions using the pipeline
pipe.predict(test_full)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  20 out of  20 | elapsed:    0.0s finished


array([3.21750000e+00, 0.00000000e+00, 1.95000000e+01, ...,
       8.13589414e+03, 5.21000000e+01, 1.23952875e+01])

In [66]:
# Importing the sample submission provided on kaggle
sample=pd.read_csv("sample_submission.csv")
sample.head()

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0


In [67]:
# Saving the predicted results
submission = pd.DataFrame(test_full.id, columns = ['id'])
submission['sales'] = pipe.predict(test_full)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  20 out of  20 | elapsed:    0.0s finished


In [68]:
# Transferring the results to a file called submission.csv
submission.to_csv('submission.csv', index = False)